In [3]:
import pandas as pd
ucsc_passage_df = pd.read_csv("passage.csv")
ucsc_passage_df


,Index,passage,gpt_prolog refactored,gpt_prolog,link,comments
0,0,The boating club exists to give community memb...,% Club definition\r\nclub(boating_club).\r\ncl...,club(boating_club).\n\nclub_purpose(boating_cl...,https://recreation.ucsc.edu/boating/community-...,NaN
1,1,Boating Club Hours\nThe Community Boating Cent...,% Boating Club hours of operation\r\nclub_hour...,"% Boating Club Hours\nclub_hours(boating_club,...",https://recreation.ucsc.edu/boating/index.html,NaN
2,2,Our boats:\nSailing vessels for weekend use cu...,"% Sailing vessels\r\nsailing_vessel(""RS Quest""...",% Facts about sailing vessels\nsailing_vessel(...,https://recreation.ucsc.edu/boating/community-...,NaN
3,3,Club Membership Fees:\n\nUCSC Student Membersh...,% Membership fees\r\nmembership_fee(ucsc_stude...,% Membership fees\nmembership_fee(ucsc_student...,https://recreation.ucsc.edu/boating/community-...,NaN
4,4,Summer Boating Classes\nEnrollment for summer ...,% Enrollment information for summer boating cl...,% Summer boating classes enrollment informatio...,https://recreation.ucsc.edu/boating/index.html,NaN
...,...,...,...,...,...,...
101,101,Newly admitted Out-of-State and International ...,NaN,% Eligibility for the Award\nautomatically_con...,https://financialaid.ucsc.edu/types-of-aid/sch...,NaN
102,102,"Two art galleries, the Eloise Pickard Smith Ga...",NaN,% Art Galleries\ngallery(eloise_pickard_smith_...,https://admissions.ucsc.edu/resources-support/...,NaN
103,103,A total of 70 semester credits (105 quarter cr...,NaN,% Credit Transfer Limits\ncredit_transfer_limi...,https://registrar.ucsc.edu/navigator/section1/...,NaN
104,104,"For fall quarter acceptance, most notices are ...",NaN,% Notification Timeline for First-Year Student...,https://admissions.ucsc.edu/resources-support/...,NaN


In [ ]:
!pip install pyarrow fastparquet huggingface_hub evaluate rouge_score langchain-openai transformers langchain langchainhub langchain-huggingface faiss-cpu langchain-community sentence_transformers

In [4]:
ucsc_qa_df = pd.read_csv("new_qa.csv", index_col=0)
ucsc_qa_df

,questions,answers,source_name,source,generated_response,source_docs
index,,,,,,
0,Who can be a member of the Boating Club?,Everyone is welcome to join! If you do not hav...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,Members of the Boating Club can include commun...,"Our club exists to give community members, UC ..."
1,What classes does the boating center offer to ...,We offer all levels of sailing (dinghy and kee...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating center offers classes for beginner...,UC Santa Cruz Community Boating Center is a co...
2,When is the boating club open?,"The club is open year-round, Saturday and Sund...",UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The boating club is open on Saturday and Sunda...,Boating Club Hours\nThe Community Boating Cent...
3,What boats are available to members?,Sailing vessels for weekend use currently incl...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,Members have access to the following sailing v...,Our boats:\nSailing vessels for weekend use cu...
4,How much does a boating membership cost?,UCSC Student Membership: $45\nNon-Student Quar...,UCSC Boating Recreation FQAs,https://recreation.ucsc.edu/boating/faq.html,The provided context does not include any info...,There are a few minor fees associated with tak...
...,...,...,...,...,...,...
107,What if I don't fulfill the general education ...,If you don't satisfy general education require...,Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,If you do not complete the IGETC or partial IG...,"By the end of the fall term prior to transfer,..."
108,What is UC TAP?,UC Transfer Admission Planner (UC TAP) is an o...,Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,"UC TAP, or the UC Transfer Admission Planner, ...",UCSC TPP\nUCSC TAG Step-by-Step\nComplete the ...
109,When are acceptance notices sent out?,"For fall quarter acceptance, notices are sent ...",Undergraduate Admissions FAQ,https://admissions.ucsc.edu/resources-support/...,The context does not specify when acceptance n...,First Year\t\nApplication Submission\n\nMay 20...


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import pandas as pd
import evaluate
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
import os
import getpass
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.llms import HuggingFaceHub

d:\AIEA Lab\LLM Logic\Agentic-RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
open_ai_llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key = openai_api_key)
open_ai_llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D84BA10FD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D84BA130A0>, root_client=<openai.OpenAI object at 0x000001D823EB3490>, root_async_client=<openai.AsyncOpenAI object at 0x000001D84BA11030>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [6]:
def get_reteriver() : 
    import pandas as pd
    ucsc_passage_df = pd.read_csv("passage.csv")
    ucsc_passge_data_loader = DataFrameLoader(ucsc_passage_df, page_content_column="passage")
    ucsc_passage_data = ucsc_passge_data_loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    docs = text_splitter.split_documents(ucsc_passage_data)
    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_documents(docs, embeddings)
    return db.as_retriever()

In [7]:
ucsc_passge_data_loader = DataFrameLoader(ucsc_passage_df, page_content_column="passage")
ucsc_passage_data = ucsc_passge_data_loader.load()
ucsc_passage_data[0]

Document(metadata={'Index ': 0, 'gpt_prolog refactored': '% Club definition\r\nclub(boating_club).\r\nclub_purpose(boating_club, "Provide access to equipment for enjoying Monterey Bay to community members, UC Santa Cruz students, faculty, and staff.").\r\n\r\n% Membership eligibility\r\neligible_for_membership(Member) :-\r\n    is_community_member(Member);\r\n    is_ucsc_student(Member);\r\n    is_ucsc_faculty(Member);\r\n    is_ucsc_staff(Member).\r\n\r\n% Club activity access\r\nhas_access_to_activity(Member, Activity) :-\r\n    eligible_for_membership(Member),\r\n    is_club_member(Member),\r\n    club_purpose(boating_club, _),\r\n    Activity = enjoy_monterey_bay.\r\n\r\n% Vessel availability\r\navailable_vessel(Vessel) :-\r\n    is_sailing_vessel(Vessel);\r\n    is_rowing_vessel(Vessel).\r\n\r\n% Supervised access to vessels\r\nsupervised_access_to_vessel(Member, Vessel) :-\r\n    has_access_to_activity(Member, enjoy_monterey_bay),\r\n    available_vessel(Vessel),\r\n    dockmaste

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = text_splitter.split_documents(ucsc_passage_data)
len(docs)

Created a chunk of size 265, which is longer than the specified 200
Created a chunk of size 1411, which is longer than the specified 200
Created a chunk of size 294, which is longer than the specified 200
Created a chunk of size 395, which is longer than the specified 200
Created a chunk of size 298, which is longer than the specified 200
Created a chunk of size 324, which is longer than the specified 200
Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 662, which is longer than the specified 200
Created a chunk of size 451, which is longer than the specified 200
Created a chunk of size 245, which is longer than the specified 200
Created a chunk of size 226, which is longer than the specified 200
Created a chunk of size 580, which is longer than the specified 200
Created a chunk of size 1970, which is longer than the specified 200
Created a chunk of size 617, which is longer than the specified 200
Created a chunk of size 312, which is longer t

526

In [9]:
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

In [10]:
index = 1
question = ucsc_qa_df["questions"][index]
print("Question: ", question)
print("Answer:", ucsc_qa_df["answers"][index])
print("Generated :", ucsc_qa_df["generated_response"][index])

Question:  What classes does the boating center offer to community members?
Answer: We offer all levels of sailing (dinghy and keelboats) and rowing classes to community members during the summer. Registration for Summer boating classes opens on March 15th. For questions about course offerings or registration, please call the following number: (831) 425-1164 or (949) 371-1448.
Generated : The boating center offers classes for beginners seeking to learn something new as well as for experienced sailors, rowers, and kayakers looking to improve their skills. These classes include 2-week junior summer camps and quarterly physical education classes for UC Santa Cruz students. The classes are designed to build confidence on the water, accommodating individuals who have never been on a boat before or those who want to build on old skills.


In [11]:
query = question
docs = db.similarity_search(query)
docs

[Document(id='c6bbb47b-3b2c-4437-8699-cabc3632fb8a', metadata={'Index ': 8, 'gpt_prolog refactored': '% Club Information\r\nclub(ucsc_community_boating_center).\r\nclub_purpose(ucsc_community_boating_center, \r\n    "Connect students and community members to the Monterey Bay through sailing, rowing, and kayaking.").\r\nclub_activities(ucsc_community_boating_center, [sailing, rowing, kayaking]).\r\n\r\n% Staff and Participant Contributions\r\nstaff_roles("Provide familiarity with equipment, skills, and water safety.").\r\nveteran_participant_roles("Share expertise from a lifetime of enjoying these sports.").\r\n\r\n% Community Connection Opportunities\r\nconnects_participants(ucsc_community_boating_center, \r\n    "Beginners and experienced participants for collaborative learning and enjoyment.").\r\noffers_classes(ucsc_community_boating_center, \r\n    "Skills improvement and water time for all experience levels.").\r\n\r\n% Available Classes and Programs\r\nclass_offering("2-week juni

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()
retriever = db.as_retriever()

source_docs = []
rag_answers = []

def format_docs(docs):
    ret_docs = "\n\n".join(doc.page_content for doc in docs)
    source_docs.append(ret_docs)
    return ret_docs

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | open_ai_llm
    | output_parser
)

for q in ucsc_qa_df['questions']:
    answer = rag_chain.invoke(question)
    rag_answers.append(answer)
    print("Question: ", question)
    print("Answer: ", answer)
    break
  

Question:  What classes does the boating center offer to community members?
Answer:  The boating center offers classes for individuals at all skill levels, including beginners who are seeking to learn something new and experienced sailors, rowers, or kayakers looking to improve their skills. Specifically, they provide a 2-week junior summer camp and quarterly physical education classes for UC Santa Cruz students. Classes are designed to accommodate those who have never been on a boat before or those who want to build on existing skills.


In [13]:
questionTest = "Am I eligible for UC SHIP if I've graduated?"
rag_chain.invoke(questionTest)

"Yes, you can be covered for one additional quarter if you've graduated and had UC SHIP in the previous quarter."

In [26]:
ucsc_qa_df[42:43]["generated_response"]

index
42    UC Santa Cruz requires students to earn a grad...
Name: generated_response, dtype: object

In [14]:

# mask = ucsc_qa_df["questions"] == "Am I eligible for UC SHIP if I've graduated?"
# matching_rows = ucsc_qa_df[mask]
ucsc_qa_df[11:15]



,questions,answers,source_name,source,generated_response,source_docs
index,,,,,,
11,How do I get UC SHIP?,All students are automatically enrolled in UC ...,UCSC Student Health Center FQAs,https://healthcenter.ucsc.edu/billing-insuranc...,"To get UC SHIP, no further action is required ...",To accept enrollment in UC SHIP no action is n...
12,How do I get my UC SHIP ID card?,Insurance ID cards are electronic. Download th...,UCSC Student Health Center FQAs,https://healthcenter.ucsc.edu/billing-insuranc...,You can get your UC SHIP ID card by downloadin...,To accept enrollment in UC SHIP no action is n...
13,Am I eligible if I opted out of UC SHIP last y...,Yes. You will be automatically enrolled in UC ...,UCSC Student Health Center FQAs,NaN,"If you opted out of UC SHIP last year, you are...",To accept enrollment in UC SHIP no action is n...
14,Am I eligible for UC SHIP if I've graduated?,"Yes, you can be covered for one additional qua...",UCSC Student Health Center FQAs,https://healthcenter.ucsc.edu/billing-insuranc...,The context does not provide information about...,Our campus also awards the UC Health Insurance...


In [5]:
src_ans_list = ucsc_qa_df['answers'].tolist()
gen_ans_list = ucsc_qa_df["generated_response"].tolist()
source_doc_list =  ucsc_qa_df["source_docs"].tolist()
question_list = ucsc_qa_df["questions"].tolist()

In [9]:

print("Source Answer: ", type(src_ans_list[0]))
print("Generated Answer: ", gen_ans_list[0])
print("Source Docs: ", source_doc_list[0])
print("Question: ", question_list[0])


Source Answer:  <class 'str'>
Generated Answer:  Members of the Boating Club can include community members, UC Santa Cruz students, faculty, and staff.
Source Docs:  Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.

Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.

Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provide

In [6]:
len(src_ans_list), len(gen_ans_list), len(source_doc_list) , len(question_list)

(112, 112, 112, 112)

In [17]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

In [18]:
bleu_results = bleu.compute(predictions=gen_ans_list, references=src_ans_list)
bleu_results

{'bleu': 0.07183728301633403,
 'precisions': [0.27566200535554897,
  0.08472012102874432,
  0.04309018159433672,
  0.02646414030692139],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0896417571729615,
 'translation_length': 6722,
 'reference_length': 6169}

In [19]:
rouge_results = rouge.compute(predictions=gen_ans_list, references=src_ans_list)
rouge_results

{'rouge1': 0.2738519832171502,
 'rouge2': 0.0910855071257653,
 'rougeL': 0.1917039087469109,
 'rougeLsum': 0.19643106678081773}

In [10]:
retrieved_contexts = [docs.split("\n\n") for docs in source_doc_list]

In [15]:
import os
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


# Define the questions, ground truth answers, generated answers, and retrieved contexts
questions = question_list
ground_truth_answers = src_ans_list
generated_answers = gen_ans_list
retrieved_contexts = [docs.split("\n\n") for docs in source_doc_list]


# Create a dataset compatible with Ragas
data = {
    "question": questions,
    "ground_truth": ground_truth_answers,
    "answer": generated_answers,
    "retrieved_contexts": retrieved_contexts,
}
dataset = Dataset.from_dict(data)

metrics = [faithfulness, answer_relevancy, context_recall, context_precision]

results = evaluate(dataset, metrics=metrics, raise_exceptions=False)


results_df = results.to_pandas()

# Display the evaluation results
print(results_df)


Evaluating: 100%|██████████| 448/448 [06:05<00:00,  1.23it/s]


                                            user_input  \
0             Who can be a member of the Boating Club?   
1    What classes does the boating center offer to ...   
2                       When is the boating club open?   
3                 What boats are available to members?   
4             How much does a boating membership cost?   
..                                                 ...   
107  What if I don't fulfill the general education ...   
108                                    What is UC TAP?   
109              When are acceptance notices sent out?   
110  What are the Cross-Campus and Simultaneous Enr...   
111    Can I talk to an adviser during a campus visit?   

                                    retrieved_contexts  \
0    [Our club exists to give community members, UC...   
1    [UC Santa Cruz Community Boating Center is a c...   
2    [Boating Club Hours\nThe Community Boating Cen...   
3    [Our boats:\nSailing vessels for weekend use c...   
4    [There a

In [16]:
average_scores = results_df[['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']].mean()

# Display the average scores
print("Average Scores:")
print(average_scores)

Average Scores:
faithfulness         0.832681
answer_relevancy     0.561461
context_recall       0.318564
context_precision    0.478964
dtype: float64
